# Object:

### Analysing comments on COVID-⁠19 Vaccines plan in Canada.

# Description:

### We will analyse the text starting with turn these comments into a meaningful format, then cleaning data by:
- Remove capital letters and replace them by lower case letters.
- Remove punctiuation.
- Remove stop words and numbers.
### AS a final steps we will use two of the topic modeling techniques, then converting comments into supervised data that we can explorate data using AMOD and counter.

# Tools:
- Numpy
- Pandas
- Sklearn
- NLTK
- RE
- Spacy
- Gensim

In [1]:
import pandas as pd
import numpy as np
import spacy
import re, nltk, spacy, gensim
nlp = spacy.load('en_core_web_sm')
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from gensim import corpora
from gensim.models.coherencemodel import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
from pprint import pprint

In [2]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [3]:
from spacy.lang.en import English
!pip install spacy && python -m spacy download en


2021-12-28 23:09:05.574981: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found

[!] As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the full



2021-12-28 23:09:05.575063: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


pipeline package name 'en_core_web_sm' instead.
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [4]:

data= pd.read_csv(r'C:\Users\sshah\OneDrive\المستندات\comment_on_plan.csv',encoding='latin-1')

In [5]:
data

,Comment
0,The vaccine plan according to liberals is in t...
1,Starting a pandemic with masks can only be wor...
2,Dont forget the Libs tossing out our PPE and ...
3,they threw out emergency stock of N95s last ye...
4,Conservatives want to take a vaccine that hasn...
...,...
1357,All is Trudeau has to do is pick up the teleph...
1358,You know for all the bad press and clear ill w...
1359,"Roll Trudeau out,the virus in Canada."
1360,As quickly as possible this could be 2022


In [6]:
data1= list(data.Comment)
data1

['The vaccine plan according to liberals is in that little red book that back in 1993 J Chrétien keep campaigning about, to this date so many liberals have added many many plans and promises in it, but no one seems to be able to find it, and now Trudeau leads the pack in there',
 'Starting a pandemic with masks can only be worn by professionals, to being among the last in the world to getting a vaccine, this is one big govt fail. We should be manufacturing our own vaccines within Canada, and N95 masks as well, enough so we can send them to every senior. incompetent.',
 'Don\x92t forget the Libs tossing out our PPE and then giving what was left to China. Fail',
 'they threw out emergency stock of N95s last year, without replacing them, ... genius move.',
 "Conservatives want to take a vaccine that hasn't been approved? No wonder they were so eager to inject bleach",
 'making stuff up',
 'drivel but thanks anyway\xa0',
 'Trump is more like JT the any Canadian conservative',
 "Its all goo

## Preprocessing:

In [7]:
documents=[]
for i in data1:
    documents.append(nlp(i))
documents

[The vaccine plan according to liberals is in that little red book that back in 1993 J Chrétien keep campaigning about, to this date so many liberals have added many many plans and promises in it, but no one seems to be able to find it, and now Trudeau leads the pack in there,
 Starting a pandemic with masks can only be worn by professionals, to being among the last in the world to getting a vaccine, this is one big govt fail. We should be manufacturing our own vaccines within Canada, and N95 masks as well, enough so we can send them to every senior. incompetent.,
 Dont forget the Libs tossing out our PPE and then giving what was left to China. Fail,
 they threw out emergency stock of N95s last year, without replacing them, ... genius move.,
 Conservatives want to take a vaccine that hasn't been approved? No wonder they were so eager to inject bleach,
 making stuff up,
 drivel but thanks anyway ,
 Trump is more like JT the any Canadian conservative,
 Its all good. The vaccines are com

In [8]:
def preprocessing(docs):
    processed_data=[]
    for e in docs:
        tokens = []
        for token in nlp(e):
            if not token.is_stop:
                tt = gensim.utils.simple_preprocess(str(token.lemma_), deacc=True)
                for i in tt: 
                    tokens.append(i)
        processed_data.append(tokens)
    return processed_data


In [9]:
preprocessed_data=preprocessing(documents)
preprocessed_data[1]

['start',
 'pandemic',
 'mask',
 'wear',
 'professional',
 'world',
 'get',
 'vaccine',
 'big',
 'govt',
 'fail',
 'manufacture',
 'vaccine',
 'canada',
 'mask',
 'send',
 'senior',
 'incompetent']

## LDA Model:

In [10]:
dictionary= corpora.Dictionary(preprocessed_data)
dt_matrix= [dictionary.doc2bow(rev) for rev in preprocessed_data]

In [11]:
lda = gensim.models.ldamodel.LdaModel(corpus=dt_matrix, num_topics=8, id2word=dictionary, passes=5)

In [12]:
lda.print_topics()

[(0,
  '0.016*"vaccine" + 0.009*"money" + 0.009*"people" + 0.008*"time" + 0.007*"go" + 0.007*"work" + 0.007*"thing" + 0.007*"need" + 0.007*"government" + 0.006*"like"'),
 (1,
  '0.014*"conservative" + 0.010*"party" + 0.009*"line" + 0.007*"far" + 0.007*"trudeau" + 0.006*"year" + 0.006*"like" + 0.005*"vaccine" + 0.005*"take" + 0.005*"find"'),
 (2,
  '0.027*"vaccine" + 0.010*"trudeau" + 0.010*"liberal" + 0.010*"know" + 0.008*"canadians" + 0.006*"like" + 0.006*"get" + 0.006*"think" + 0.005*"plan" + 0.005*"health"'),
 (3,
  '0.047*"possible" + 0.046*"trudeau" + 0.046*"quickly" + 0.035*"say" + 0.014*"vaccine" + 0.012*"canada" + 0.009*"plan" + 0.008*"incompetence" + 0.008*"answer" + 0.007*"believe"'),
 (4,
  '0.036*"vaccine" + 0.021*"canada" + 0.012*"health" + 0.011*"approve" + 0.010*"country" + 0.009*"trudeau" + 0.008*"quickly" + 0.008*"possible" + 0.008*"year" + 0.007*"line"'),
 (5,
  '0.053*"vaccine" + 0.021*"canada" + 0.012*"government" + 0.010*"covid" + 0.009*"country" + 0.009*"canadians

## Evaluate LDA.

In [13]:
cohrence_lda_model= CoherenceModel(model=lda, texts= preprocessed_data, dictionary= dictionary, coherence='c_v')
cohrence_lda= cohrence_lda_model.get_coherence()
print(f'\n Cohrence score: {cohrence_lda}')


 Cohrence score: 0.3229744829667019


# determining best number of topics for LDA Model.

In [14]:
def compute_cohrence_values(dictionary, corpus, texts, limit, start=2, step=1):
    coherence_values=[]
    model_list=[]
    for num_topics in range(start, limit, step):
        model= gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics= num_topics, id2word= dictionary)
        model_list.append(model)
        coherence_model= CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherence_model.get_coherence())
    return model_list, coherence_values

In [15]:
model_list, coherence_values= compute_cohrence_values(dictionary= dictionary, corpus= dt_matrix, texts= preprocessed_data, limit=9, start= 2, step=1)

In [16]:
start=2
limit=9
step=1

x= range(start,limit,step)

In [17]:
for topic, cv in zip(x, coherence_values):
    print('Number of topics:', topic, 'has coherence score:', round(cv,4))

Number of topics: 2 has coherence score: 0.3064
Number of topics: 3 has coherence score: 0.325
Number of topics: 4 has coherence score: 0.3044
Number of topics: 5 has coherence score: 0.3155
Number of topics: 6 has coherence score: 0.3408
Number of topics: 7 has coherence score: 0.3427
Number of topics: 8 has coherence score: 0.3313


## From the previous, we found out that the best model will have 7 topics

In [18]:
optimal_model= model_list[5]
model_topics= optimal_model.show_topics(formatted=False)
optimal_model.print_topics(num_words=10)

[(0,
  '0.026*"vaccine" + 0.016*"canada" + 0.014*"quickly" + 0.014*"possible" + 0.007*"get" + 0.006*"approve" + 0.006*"liberal" + 0.006*"approval" + 0.006*"distribution" + 0.005*"people"'),
 (1,
  '0.017*"vaccine" + 0.012*"canada" + 0.011*"think" + 0.010*"country" + 0.009*"million" + 0.007*"say" + 0.007*"trudeau" + 0.007*"plan" + 0.005*"dose" + 0.005*"pm"'),
 (2,
  '0.018*"vaccine" + 0.017*"trudeau" + 0.016*"canada" + 0.012*"government" + 0.008*"possible" + 0.008*"health" + 0.008*"quickly" + 0.008*"say" + 0.007*"start" + 0.007*"go"'),
 (3,
  '0.016*"vaccine" + 0.013*"know" + 0.013*"possible" + 0.012*"quickly" + 0.009*"year" + 0.008*"trudeau" + 0.007*"say" + 0.007*"canada" + 0.006*"come" + 0.006*"china"'),
 (4,
  '0.047*"vaccine" + 0.012*"people" + 0.010*"trudeau" + 0.008*"need" + 0.008*"canada" + 0.007*"say" + 0.007*"canadians" + 0.007*"think" + 0.007*"way" + 0.006*"like"'),
 (5,
  '0.031*"trudeau" + 0.017*"vaccine" + 0.016*"say" + 0.015*"time" + 0.015*"quickly" + 0.014*"possible" + 0.

## Visualaization:

In [19]:
visualaization= gensimvis.prepare(optimal_model,dt_matrix,dictionary)
visualaization

C:\Users\sshah\anaconda3\envs\jupyterr\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.000761 -0.032065       1        1  23.751103
4     -0.036486  0.020570       2        1  17.060941
5     -0.026830 -0.054871       3        1  14.235103
2     -0.025788 -0.052532       4        1  13.869001
0      0.042332  0.030181       5        1  12.165836
1     -0.046282  0.083524       6        1   9.707617
3      0.093815  0.005193       7        1   9.210399, topic_info=        Term        Freq       Total Category  logprob  loglift
15   trudeau  230.000000  230.000000  Default  30.0000  30.0000
85      know   77.000000   77.000000  Default  29.0000  29.0000
113    think   59.000000   59.000000  Default  28.0000  28.0000
16   vaccine  418.000000  418.000000  Default  27.0000  27.0000
148     time   84.000000   84.000000  Default  26.0000  26.0000
..       ...         ...         ...      ...      ...      ...
105  believe    4.742671   33.563638   Topic7  -5.7649   0.4280
75    people    5.903502   89.562130   Topic7  -5.5460  -0.3345
46   approve    5.284566   62.909628   Topic7  -5.6567  -0.0921
777  country    5.122883   71.543085   Topic7  -5.6878  -0.2517
113    think    4.909972   59.471095   Topic7  -5.7303  -0.1094

[586 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
1005      1  0.772286  abandon
0         2  0.101241     able
0         3  0.303724     able
0         4  0.404966     able
0         5  0.101241     able
...     ...       ...      ...
45        3  0.142928     year
45        4  0.079404     year
45        5  0.142928     year
45        6  0.063524     year
45        7  0.206452     year

[1307 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 5, 6, 3, 1, 2, 4])

### now it's clear that we didn't achieve the high objective results with LDA model, so we'll try with another model.

# CorEx Model:

In [20]:
!pip install corextopic
!pip install networkx
from corextopic import corextopic as ct
from corextopic import vis_topic as vt

In [21]:
vectorizer2 = CountVectorizer(max_features=20000,
                             stop_words='english', token_pattern="\\b[a-z][a-z]+\\b",
                             binary=True)

doc_word = vectorizer2.fit_transform(data1)
words = list(np.asarray(vectorizer2.get_feature_names()))

C:\Users\sshah\anaconda3\envs\jupyterr\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [22]:
topic_model = ct.Corex(n_hidden=4, words=words, seed=1)
topic_model.fit(doc_word, words=words, docs=data1)

In [23]:
topics = topic_model.get_topics()
for n,topic in enumerate(topics):
    topic_words,_,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))
    
    categories = ['Vaccine.plan', 'Healthcare.in.Canada', 
              'Canadian.goverment', 'Trudeau.and.the.liberal.party.of.Canada']

0: evidence,canada,control,multiple,countries,billion,pandemic,delivering,meant,later
1: vaccine,going,people,weeks,distribution,signed,receive,cansino,likely,health
2: quickly,possible,says,trudeau,incompetence,plan,doses,liberals,million,government
3: federal,gone,used,getting,information,spending,angry,tired,clinical,covid


In [24]:
predictions = pd.DataFrame(topic_model.predict(doc_word), columns=['topic'+str(i) for i in range(4)])
predictions

,topic0,topic1,topic2,topic3
0,True,False,True,False
1,False,False,False,True
2,False,False,False,True
3,False,False,True,False
4,False,False,False,False
...,...,...,...,...
1357,False,True,False,False
1358,False,False,True,True
1359,False,False,False,False
1360,False,False,True,False


In [25]:
topic_model.fit(doc_word, words=words, docs=data1, 
                anchors=[['plan', 'decide'], ['healthcare','health','care','children'],['government','country','citizen','decision'],['canada','US','liberal','pay']], anchor_strength=10)

topics = topic_model.get_topics()
for n,topic in enumerate(topics):
    topic_words,_,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))


0: plan,decide,low,distribution,version,citizens,falling,impact,sidelines,statistics
1: health,care,healthcare,children,workers,provinces,oil,jobs,approval,vaccines
2: government,country,people,example,private,playing,domestically,current,ottawa,wrong
3: canada,liberal,pay,vaccine,approve,foreign,production,chinese,blocked,domestic


In [26]:
topics

[[('plan', 1.414018340081795, 1.0),
  ('decide', 0.04465564096072224, 1.0),
  ('low', 0.011284747795852227, 1.0),
  ('distribution', 0.010662170696289306, 1.0),
  ('version', 0.009325814168233377, 1.0),
  ('citizens', 0.008482299996407407, 1.0),
  ('falling', 0.006892214175120324, 1.0),
  ('impact', 0.006892214175120324, 1.0),
  ('sidelines', 0.006892214175120324, 1.0),
  ('statistics', 0.006892214175120324, 1.0)],
 [('health', 2.2681551570942555, 1.0),
  ('care', 0.8408324662832578, 1.0),
  ('healthcare', 0.24318029647287515, 1.0),
  ('children', 0.04512429827107381, 1.0),
  ('workers', 0.01782684512977439, 1.0),
  ('provinces', 0.012475109063445785, 1.0),
  ('oil', 0.011884888049206914, 1.0),
  ('jobs', 0.011884888049206914, 1.0),
  ('approval', 0.011784753448307723, 1.0),
  ('vaccines', 0.011072715596201713, 1.0)],
 [('government', 2.4623831879410076, 1.0),
  ('country', 0.909519504182787, 1.0),
  ('people', 0.0160248320540512, 1.0),
  ('example', 0.013534736935286626, 1.0),
  ('pri

In [27]:
predictions = pd.DataFrame(topic_model.predict(doc_word), columns=['topic'+str(i) for i in range(4)])
predictions

,topic0,topic1,topic2,topic3
0,True,False,False,False
1,False,False,False,True
2,False,False,False,False
3,False,False,False,False
4,False,False,False,False
...,...,...,...,...
1357,False,False,False,False
1358,False,False,True,False
1359,False,False,False,True
1360,False,False,False,False


## Converting data into supervised data:

In [28]:
data['spacy_doc'] = list(nlp.pipe(data.Comment))

In [29]:
data['index']= range(0,1362)

In [30]:
predictions['index']= range(0,1362)

In [31]:
spacy_data=pd.merge(data,predictions, on= ['index','index'])

In [32]:
spacy_data.rename(columns = {'topic0':'covid_plan', 'topic1':'healthcare', 'topic2':'canadian_government','topic3':'liberal_party'}, inplace = True)

In [34]:
spacy_data

,Comment,spacy_doc,index,covid_plan,healthcare,canadian_government,liberal_party
0,The vaccine plan according to liberals is in t...,"(The, vaccine, plan, according, to, liberals, ...",0,True,False,False,False
1,Starting a pandemic with masks can only be wor...,"(Starting, a, pandemic, with, masks, can, only...",1,False,False,False,True
2,Dont forget the Libs tossing out our PPE and ...,"(Dont, forget, the, Libs, tossing, out, our, ...",2,False,False,False,False
3,they threw out emergency stock of N95s last ye...,"(they, threw, out, emergency, stock, of, N95s,...",3,False,False,False,False
4,Conservatives want to take a vaccine that hasn...,"(Conservatives, want, to, take, a, vaccine, th...",4,False,False,False,False
...,...,...,...,...,...,...,...
1357,All is Trudeau has to do is pick up the teleph...,"(All, is, Trudeau, has, to, do, is, pick, up, ...",1357,False,False,False,False
1358,You know for all the bad press and clear ill w...,"(You, know, for, all, the, bad, press, and, cl...",1358,False,False,True,False
1359,"Roll Trudeau out,the virus in Canada.","(Roll, Trudeau, out, ,, the, virus, in, Canada...",1359,False,False,False,True
1360,As quickly as possible this could be 2022,"(As, quickly, as, possible, this, could, be, 2...",1360,False,False,False,False


In [35]:
type(spacy_data.spacy_doc[0])

spacy.tokens.doc.Doc

In [36]:
covid_plan_reviews = spacy_data[spacy_data.covid_plan==True]
healthcare_reviews = spacy_data[spacy_data.healthcare==True]
canadian_government_reviews = spacy_data[spacy_data.canadian_government==True]
liberal_party_reviews = spacy_data[spacy_data.liberal_party==True]

## EDA with Spacy(Amods & Counter):

In [37]:
from spacy.symbols import amod

In [38]:
def get_amods(noun, ser):
    amod_list = []
    for doc in ser:
        for token in doc:
            if (token.text) == noun:
                for child in token.children:
                    if child.dep == amod:
                        amod_list.append(child.text.lower())
    return sorted(amod_list)

def amods_by_sentiment(noun):
    print(f"Adjectives describing {str.upper(noun)}:\n")
    
    print("\nCovid plan topic:")
    pprint(get_amods(noun, covid_plan_reviews.spacy_doc))
    
    print("\nHealthcare topic:")
    pprint(get_amods(noun, healthcare_reviews.spacy_doc))
    print("\n Canadian government topic:")
    pprint(get_amods(noun, canadian_government_reviews.spacy_doc))
    print("\nLiberal party topic:")
    pprint(get_amods(noun, liberal_party_reviews.spacy_doc))
   

In [45]:
amods_by_sentiment('Canada')


Adjectives describing CANADA:


Covid plan topic:
[]

Healthcare topic:
['same']

 Canadian government topic:
['same']

Liberal party topic:
['same']


In [40]:
covidplan_adj = [token.text.lower() for doc in covid_plan_reviews.spacy_doc for token in doc if token.pos_=='ADJ']
healthcare_adj = [token.text.lower() for doc in healthcare_reviews.spacy_doc for token in doc if token.pos_=='ADJ']
canadiangovernment_adj = [token.text.lower() for doc in canadian_government_reviews.spacy_doc for token in doc if token.pos_=='ADJ']
liberal_party_adj = [token.text.lower() for doc in liberal_party_reviews.spacy_doc for token in doc if token.pos_=='ADJ']


covidplan_noun = [token.text.lower() for doc in covid_plan_reviews.spacy_doc for token in doc if token.pos_=='NOUN']
healthcare_noun = [token.text.lower() for doc in healthcare_reviews.spacy_doc for token in doc if token.pos_=='NOUN']
canadiangovernment_noun = [token.text.lower() for doc in canadian_government_reviews.spacy_doc for token in doc if token.pos_=='NOUN']
liberal_party_noun = [token.text.lower() for doc in liberal_party_reviews.spacy_doc for token in doc if token.pos_=='NOUN']



In [41]:
from collections import Counter

In [42]:
Counter(canadiangovernment_adj).most_common(10)

[('possible', 38),
 ('more', 13),
 ('first', 11),
 ('other', 11),
 ('many', 9),
 ('only', 8),
 ('conservative', 8),
 ('covid', 8),
 ('much', 7),
 ('own', 7)]

In [43]:
Counter(liberal_party_noun).most_common(10)

[('vaccine', 120),
 ('vaccines', 40),
 ('government', 35),
 ('countries', 22),
 ('time', 20),
 ('plan', 17),
 ('approval', 17),
 ('%', 17),
 ('production', 17),
 ('people', 16)]

In [44]:
Counter(healthcare_adj).most_common(10)

[('possible', 28),
 ('other', 16),
 ('safe', 11),
 ('good', 10),
 ('more', 9),
 ('first', 8),
 ('much', 8),
 ('long', 7),
 ('pandemic', 7),
 ('only', 7)]

In [164]:
Counter(canadiangovernment_noun).most_common(10)

[('government', 99),
 ('vaccine', 88),
 ('people', 37),
 ('country', 37),
 ('vaccines', 23),
 ('time', 19),
 ('plan', 14),
 ('production', 13),
 ('world', 12),
 ('distribution', 12)]